In [1]:
import pandas as pd
import os

# Load the CSV
csv_path = "cleaned_foid.csv"
df = pd.read_csv(csv_path)

In [5]:
# Create a label to class_id mapping
label_map = {label: idx for idx, label in enumerate(df['label'].unique())}

# Directory where YOLO txt files will be saved
output_dir = "labels/"
os.makedirs(output_dir, exist_ok=True)

# Group annotations by image
for filename, group in df.groupby('img_id'):
    label_lines = []
    for _, row in group.iterrows():
        class_id = label_map[row['label']]
        x_min, y_min, x_max, y_max = row['x_min'], row['y_min'], row['x_max'], row['y_max']
        img_width, img_height = row['width'], row['height']

        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Write to corresponding YOLO format .txt file
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    with open(os.path.join(output_dir, txt_filename), "w") as f:
        f.write("\n".join(label_lines))